plot_change.py: Generate significance of electrode weight changes at neuroanatomical sites

## Pseudocode:
    . Import
    . Define functions
    . Input symptom, mode, perdur
    . Load Parameters
    . For Loop through seizures

## Import

In [1]:
import csv
import os
import pathlib
import numpy as np
from scipy.io import loadmat
from scipy.signal import butter,filtfilt

#
import pandas as pd
# from plot_change_module import *

In [2]:
neuroanat_list = ['frontalpole', #FRONTAL LOBE
    'parstriangularis',
    'parsopercularis',
    'parsorbitalis',
    'rostralmiddlefrontal',
    'caudalmiddlefrontal',
    'lateralorbitofrontal',
    'superiorfrontal',
    'medialorbitofrontal',
    'precentral',
    'postcentral', # PARIETAL LOBE
    'inferiorparietal',   
    'superiorparietal',
    'supramarginal',
    'temporalpole', # TEMPORAL LOBE
    'middletemporal',
    'superiortemporal',
    'inferiortemporal',
    'parahippocampal',               
    'Right-Hippocampus',
    'Left-Hippocampus',
    'Right-Amygdala',
    'Left-Amygdala',
    'entorhinal',
    'bankssts',
    'fusiform', # OCCIPITAL LOBE                
    'lingual',
    'Right-Inf-Lat-Vent', # OTHER
    'Right-Cerebral-White-Matter',
    'Left-Cerebral-White-Matter',
    'Right-choroid-plexus',
    'Right-Putamen',
    'Right-VentralDC'];

## Define Functions

In [3]:
def ll_sort(LL,anat,neuroanat_list):
    anat_col = anat.iloc[:,3]
    
    a_count = 0
    s_count = 0
    n_count = 0
    y_count = 0
    
    n_list = []
    y_list = []
    LL_s = np.empty([np.shape(anat)[0],np.shape(LL)[1]])
    
    for n_a in neuroanat_list:
        # print(a_count)
        a_count = 0
        for a_c in anat_col:
            if np.size(a_c) > 0: 
                s_count += 1
                if n_a == a_c[0]:
#                 #collect ll data
                    n_list.append(n_a) #collect all neuroanat str
                    LL_s[n_count,:] = LL[a_count,:]
                    n_count += 1

                    #sort into unique y tick labels
                    if n_a not in y_list:
                        y_list.append(n_a) #collect only non-repeating neuroanat str  
                        y_count += 1
            a_count += 1
    
    return LL_s                     
    # print(a_count)
    # print(s_count)
    # print(n_count)
    # print(y_count)


In [4]:
def ll_transform(llw,fs,d,bl_start,bl_stop):
    L = int(np.round(llw * fs) - 1) # number of samples to calculate line length
    LL = np.empty(np.shape(d))
    LL[:] = np.NaN    
    
    
    
    sample_bl_start = max([0,1])

    if bl_start == 0:
        sample_bl_start = 0
    elif bl_start > 0:
        sample_bl_start = (fs*bl_start)[0] #0 because it's python not matlab
    sample_bl_end = (fs*bl_stop)[0]-1
    
    print(sample_bl_start)
    print(sample_bl_end)
    
    # print(np.shape(d)[0])
    # Line-length transform
    
    for row_1 in range(90,100):
    # for row_1 in range(0,np.shape(d)[0]-1):
        LL_list = []

        for col_1 in range(sample_bl_start,np.shape(d)[1]-L):
        # for col_1 in range(0,np.shape(d)[1]-L):
            LL[row_1,col_1] = np.sum(np.abs(np.diff(d.iloc[row_1,col_1:col_1+L])))
            
        LL_list = LL[row_1,sample_bl_start:np.shape(d)[1]-L].tolist()
        
        if np.isnan(LL_list).all():
        #     nan_count += 1
             print('nan')
        else:
             print('real or zero')

            
#             if np.count_nonzero(LL[row_1,sample_bl_start:sample_bl_end]) == 0:
#             #     print('garghh!')
#         # print(LL[row_1,sample_bl_start:sample_bl_end])
#                 LL_nanmean = np.nanmean(LL[row_1,sample_bl_start:sample_bl_end])
#                 LL_nanstd = np.nanstd(LL[row_1,sample_bl_start:sample_bl_end])

#                 LL[row_1,:] = (LL[row_1,:] - LL_nanmean)/LL_nanstd
#                 print(LL[row_1,:])
#             else:
#                 print('***********')
#                 print('zero or nan')
#                 LL[row_1,:] = LL[row_1,:]
#                 print(LL[row_1,:])
#                 print('***********')
    # Normalize LL (to baseline period) as z-scores, "zLL"
    # for row_2 in range(0,np.shape(d)[0]):    
    #     LL_nanmean = np.nanmean(LL[row_2,sample_bl_start:sample_bl_end])
    #     LL_nanstd = np.nanstd(LL[row_2,sample_bl_start:sample_bl_end])

#         LL[row_2,:] = (LL[row_2,:] - LL_nanmean)/LL_nanstd
    # print(LL[90:99,:])
    # return LL


In [5]:
def open_xl(xl_name): 
    # df = pd.read_excel(xl_name)
    # Ndimensions = df.shape #row, col
    # params_list = df.values.tolist()
    
    df = pd.read_excel(xl_name, index_col=0, engine='openpyxl')

    return df


In [6]:
def filt_perdur(sx_input,mx_input,perdur_input,d,sem_df,fs):
    d_t = d.transpose()
    
    butter_array = np.array([1,(round(fs[0]/2)-1)])
    b,a = butter(2,butter_array/(fs[0]/2),btype='bandpass',output='ba')
    
    mx_index = sem_df.index[sem_df[sx_input]==int(mx_input)][0] # first onset of symptom in ppEEG time 
    onset_mx = round((mx_index/5)*fs[0]) #adjusted for ppEEG time

    ppEEG_perdur = int(perdur_input) * fs[0]

    # onset_series = d.iloc[onset_mx]
    before_series = d_t.iloc[onset_mx - ppEEG_perdur]
    after_series = d_t.iloc[onset_mx + ppEEG_perdur]

    # onset_array = onset_series.to_numpy()
    before_array = before_series.to_numpy()
    after_array = after_series.to_numpy()

    # onset_array = filtfilt(b,a,~np.isnan(onset_array),axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)) # d(nns,:) = filtfilt(b,a,d(nns,:)')'; 
    before_array = filtfilt(b,a,~np.isnan(before_array),axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)).transpose() # d(nns,:) = filtfilt(b,a,d(nns,:)')'; 
    after_array = filtfilt(b,a,~np.isnan(after_array),axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)).transpose() # d(nns,:) = filtfilt(b,a,d(nns,:)')'; 

    return before_array, after_array


In [7]:
def load_elecs_anat(pt_path):
                
    #load anatomy and electrode matrix
    os.chdir(pt_path + 'Imaging/elecs')
    e_mat = loadmat('clinical_elecs_all.mat')
    anatomy = e_mat['anatomy']
    
    # u_anat = anatomy.unique()
    # print(u_anat)
    # em = e_mat['elecmatrix']
    # em_row = np.shape(anatomy)[0]
    
    return anatomy


In [8]:
def get_params(df_params,pt):
    params_bl_start = df_params.loc[pt]['BLstart']
    params_bl_stop = df_params.loc[pt]['BLstop']
    params_llw = df_params.loc[pt]['llw']

    return params_bl_start, params_bl_stop, params_llw

In [9]:
def get_ptsz(avg_change_path): #Load list of patient and seizure names
    os.chdir(avg_change_path)
    
    df_params = open_xl('OPSCEAparams.xlsx') # Ndimensions and params_list

    with open('sz_name_list.csv','r') as sz_names:
        sz_names_list = list(csv.reader(sz_names))
        sz_rows = np.size(sz_names_list,0) #find row num 
        pt_data = []
        sz_data= []
        for r in range(0,sz_rows):  #collect symptom data for specfic seizure in list
            pt_data.append(sz_names_list[r][0]) 
            sz_data.append(sz_names_list[r][1]) 
    return pt_data, sz_data, df_params

## Input symptom, mode, and duration

In [10]:
sx_input = 'lhx'
mx_input = '2'
perdur_input = '10'

## For loop through patients in list matching seizure

In [11]:
avg_change_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/OPSCEADATA/avg_change_folders/'

pt_data,sz_data,df_params = get_ptsz(avg_change_path) #Ehsan

# for neuro_struc in neuroanat_list:

#For loop through patients
for i in range(len(pt_data)):  #Ehsan
    pt_i = pt_data[i]          #Ehsan
    sz_name = sz_data[i]       #Ehsan

    pt_path = avg_change_path + pt_i + '/'
    os.chdir(pt_path)
    pt_dir = os.listdir(pt_path)

    for sz_name in sz_data: 

        if sz_name in pt_dir: #patients in list matching seizure
 
            print(sz_name)
    

            #load data
            
            # PARAMS
            blstart, blstop, llw = get_params(df_params,pt_i) # 2 = params_llw

            # ANAT 
            anatomy = load_elecs_anat(pt_path)
            anat = pd.DataFrame(anatomy)
            del anatomy
            
            a_col = np.shape(anat)[0]
            # print(a_col)
            

            # SEMIOLOGY
            sz_path = pt_path + sz_data[i]
            os.chdir(sz_path)
            
            sem_df = pd.read_csv(sz_name + '_mat.csv',usecols = [sx_input]) #load semiology matrix .csv

            
            # FS & PPEEG
            sz_mat = loadmat(sz_name +'.mat') #load frame speed and electrode matrix    
            fs = sz_mat['fs'].flatten()
            ppEEG = sz_mat['ppEEG']

            d = pd.DataFrame(ppEEG)
            del ppEEG
            
            new_d = d.drop(index=d.iloc[a_col:,:].index) # shorten d to row length of anat
            del d            
                        
            #remove EKG lines at end of anatomy + ppEEG?


            try:
                filt_before, filt_after = filt_perdur(sx_input,mx_input,perdur_input,new_d,sem_df,fs) #creates 2 arrays of before and after seizure onset and filters
                # try:
                #     ll_transform(llw,fs,new_d,blstart,blstop)
                #     # LL = ll_transform(llw,fs,new_d,blstart,blstop)
                #     # try:
                #     #     LL_s = ll_sort(LL,anat,neuroanat_list)
                #     #     # print(LL_s)
                #     # except:
                #     #     print('no LL_s')
                # except:
                #     print('no ll')
            except:
                print('no filt')
            
            ll_transform(llw,fs,new_d,blstart,blstop)
            
#             try:
#                 LL_s = ll_sort(LL,anat,neuroanat_list)
#             except:
#                 print('no LL_s')
                
            # LL_s = ll_sort(LL,anat,neuroanat_list)


EC91_03
0
5119
nan
real or zero
real or zero
nan
real or zero
real or zero
real or zero
real or zero
real or zero
real or zero
EC96_01
512
4607
nan
real or zero
real or zero
real or zero
real or zero
real or zero
real or zero
real or zero
real or zero


KeyboardInterrupt: 